# Installation

Check that the NVIDIA compiler is present.

In [99]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


## CUDA Utilities

In [100]:
%%writefile cuda_stuff.cuh
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#ifndef cuda_stuff_H
#define cuda_stuff_H

/* transform matrix index to vector offset
   Since CUDA uses column major, 
   nb_rows = number of rows */
#define IDX2C(i,j,nb_rows) (((j)*(nb_rows))+(i))
 
//MACRO TO DEBUGG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
/** Error checking for use with CUDA Dynamic Parallelism */
/*
#define cdpErrchk(ans) { cdpAssert((ans), __FILE__, __LINE__); }
__device__ void cdpAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      printf("GPU kernel assert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) assert(0);
   }
}
*/
void device_synchronize();

#endif


Overwriting cuda_stuff.cuh


In [101]:
%%writefile cuda_stuff.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include "cuda_stuff.cuh"

void device_synchronize(){
    gpuErrchk(cudaDeviceSynchronize());
}

Overwriting cuda_stuff.cu


# The fmatrix Data Structure

Similar to cuBLAS, a 2D matrix is represented by a 1D array that stores one column of the matrix after another (this is called _linearization_).
The `fmatrix` structure contains a `data` pointer to such an array, which is 100% compatible with cuBLAS. In addition, it stores the number of columns and rows in the matrix.

In [102]:
%%writefile fmatrix.cuh
#ifndef fmatrices_H
#define fmatrices_H
#include "cuda_stuff.cuh" // for IDX2C

////////////////////////////////////////
// basic data structure and access macro
////////////////////////////////////////
typedef struct {
    float* data;
    int cols;
    int rows;
} fmatrix;

/** Access element (i,j) of matrix M 
 *
 *  Usage example:
 *  For computing A = B^T + C), loop over i and j with:
 *    getfm(A,i,j) = getfm(B,j,i) + getfm(C,i,j);
 **/
#define getfm(M,i,j) (M.data[IDX2C(i,j,M.rows)])

////////////////////////////////////////
// utility functions
////////////////////////////////////////
/** Returns the number of elements in the matrix.
 *
 *  Useful for computing, e.g., the size
 *  of a 1D-vector that contains the same numbers.
 */
 __host__
 __device__
int fmatrix_elements(fmatrix mat);

/** Returns the memory occupied by the matrix elements in bytes
 *  (not including the variables in the struct mat).
 *
 *  Useful for allocating memory for the data.
 */
 __host__
 __device__
int fmatrix_size(fmatrix mat);

/** Assert that the matrix is coherent: all fields nonzero. */
 __host__
 __device__
void fmatrix_assert(fmatrix mat);

////////////////////////////////////////
// Create, copy, destroy
////////////////////////////////////////
/** Allocate memory on host */
fmatrix fmatrix_create_on_host(int rows, int cols);

/** Allocate memory on device */
fmatrix fmatrix_create_on_device(int rows, int cols);

/** Create a matrix representing columns [a,b) of M. 
 *  Note that the new matrix uses a pointer to the
 *  data of M. The data is not copied to a new location.
 *  If M is destroyed, this matrix is useless.
 */
fmatrix fmatrix_subcolumns(fmatrix M, int a, int b);

/** Copy data from matrix on device to host 
 *  (no memory allocation). */
void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device);

/** Copy data from matrix on host to device
 *  (no memory allocation). */
void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device);

/** Copy matrix from device to host, allocating new memory. */
fmatrix fmatrix_copy_to_host(fmatrix mat_device);

/** Copy matrix from host to device, allocating new memory. */
fmatrix fmatrix_copy_to_device(fmatrix mat_host);

/** Free data memory on host. 
 *  This zeros out the data pointer of the fmatrix struct,
 *  so a pointer is required. */
void fmatrix_free_on_host(fmatrix* mat);

/** Free data memory on device. 
 *  This zeros out the data pointer of the fmatrix struct,
 *  so a pointer is required. */
void fmatrix_free_on_device(fmatrix* mat);

////////////////////////////////////////
// Input and Output
////////////////////////////////////////

/** Print the first nb rows of the matrix mat
 *  on the host. 
 *  If nb<0, print all rows. 
 */
 __host__
 __device__
void fmatrix_print(fmatrix mat, int nb=-1);

/** Print the first nb rows of the matrix mat
 *  on the device. 
 *  If nb<0, print all rows. 
 *
 *  This version copies the matrix to host first.
 */
void fmatrix_device_print(fmatrix mat, int nb=-1);

/** Print a matrix to a csv file. 
 *
 *  This version copies the matrix to host first.
 */
void fmatrix_device_to_csv(const char* filename, fmatrix mat);

/** Read a matrix from a csv file. 
 *
 *  This version creates the matrix on the host first.
 */
fmatrix fmatrix_device_from_csv(const char* filename);

////////////////////////////////////////
// Useful
////////////////////////////////////////

/** Create a matrix with random values between -1 and 1
 *  on the device */
fmatrix fmatrix_create_random_on_device(int rows, int cols);

#endif


Overwriting fmatrix.cuh


In [103]:
%%writefile fmatrix.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <curand.h>
#include <curand_kernel.h>
#include "cuda_stuff.cuh"
#include "fmatrix.cuh"

// for reading CSV files, we use some C++
#include <iostream>
#include <iomanip>
#include <fstream>
#include <string>

int fmatrix_elements(fmatrix mat) {
     return mat.cols*mat.rows;
}

int fmatrix_size(fmatrix mat) {
    fmatrix_assert(mat);
     return fmatrix_elements(mat) * sizeof(mat.data[0]);
}

void fmatrix_assert(fmatrix mat) {
    assert(mat.data);
    assert(mat.cols);
    assert(mat.rows);
}

fmatrix fmatrix_create_on_host(int rows, int cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    mat.data = (float*)malloc(fmatrix_size(mat)); 
    assert(mat.data);
    return mat;
}

fmatrix fmatrix_create_on_device(int rows, int cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    gpuErrchk( 
        cudaMalloc((void **)&(mat.data), fmatrix_size(mat)) 
    );
    return mat;
}

void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk( 
        cudaMemcpy( mat_device.data, mat_host.data, 
                   fmatrix_size(mat_host), 
                   cudaMemcpyHostToDevice 
                   )
        );
}

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_host.data, mat_device.data,  
                   fmatrix_size(mat_device), 
                   cudaMemcpyDeviceToHost 
                   )
        );
}

fmatrix fmatrix_copy_to_host(fmatrix mat_device) {
    fmatrix_assert(mat_device);
    fmatrix mat_host = fmatrix_create_on_host(mat_device.rows, mat_device.cols);
    fmatrix_data_to_host(mat_host,mat_device);
    return mat_host;
}

fmatrix fmatrix_copy_to_device(fmatrix mat_host) {
    fmatrix_assert(mat_host);
    fmatrix mat_device = fmatrix_create_on_device(mat_host.rows, mat_host.cols);
    fmatrix_data_to_device(mat_host,mat_device);
    return mat_device;
}

/** We could do it like this, but it would not set our pointer M.data to 0. 
... fmatrix_free_on_host(M)
void fmatrix_free_on_host(fmatrix mat) {
    fmatrix_assert(mat);  
  free(mat.data);
  mat.data = 0;
  mat.cols = 0;
  mat.rows = 0;
}
*/

void fmatrix_free_on_host(fmatrix* mat) {
    fmatrix_assert(*mat);  
  free(mat->data);
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_free_on_device(fmatrix* mat) {
    fmatrix_assert(*mat);  
  gpuErrchk(cudaFree(mat->data));
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

fmatrix fmatrix_subcolumns(fmatrix M, int a, int b) {
    fmatrix_assert(M);  
    fmatrix A = { 
        .data = &getfm(M,0,a),  
        .cols = b-a,
        .rows = M.rows 
    };
    fmatrix_assert(A);  
    return A;
}


__host__
__device__
void fmatrix_print(fmatrix mat, int nb){
    if (nb<0 || nb > mat.rows) {
        nb = mat.rows;
    }
    printf("[\n");
    for (int i = 0 ; i < nb; i++){
      for (int j = 0 ; j<mat.cols; j++){
        printf("%f", getfm(mat,i,j));
        if (j+1<mat.cols) {
          printf(",\t");
        }
      }
      if (i+1<nb) {
        printf(";\n");
      }
    }
    if (nb < mat.rows) {
      printf("\n...\n");
    }
  printf("\n]\n");
}

void fmatrix_device_print(fmatrix mat, int nb){
   // allocate copy
   fmatrix tmp = fmatrix_copy_to_host(mat);
   fmatrix_print(tmp,nb);
   fmatrix_free_on_host(&tmp);
}

void fmatrix_device_to_csv(const char* filename, fmatrix mat) {
  // Open file
  FILE* fp = fopen(filename, "w");
  // allocate copy
  fmatrix tmp = fmatrix_copy_to_host(mat);
  for (int i = 0 ; i < tmp.rows; i++){
    for (int j = 0 ; j<tmp.cols; j++){
      // Note: %.15g gives 15 significant digits (full double precision)
      fprintf(fp,"%.15g", getfm(tmp,i,j));
      if (j+1<tmp.cols) {
        fprintf(fp,",");
      }
    }
    fprintf(fp,"\n");
  }
  fmatrix_free_on_host(&tmp);
  // Close file
  fclose(fp);
}

__global__
void fmatrix_create_random_on_device_kernel(fmatrix M) {
    // choose a seed (here: the same each launch)
    unsigned long seed = 0;
    int sequence = 0;
    // first, initialize the random numbers
    curandState state;
    curand_init(seed, sequence, 0, &state);
    for (int i = 0; i < fmatrix_elements(M); ++i) {
        // curand_uniform creates numbers between 0 and 1
        M.data[i] = (curand_uniform(&state)-0.5)*2.0;
    }
}

fmatrix fmatrix_create_random_on_device(int rows, int cols) {
    // Create an uninitialized matrix on the device
    fmatrix M = fmatrix_create_on_device(rows,cols);
    // Call a kernel with a single thread to fill the values
    fmatrix_create_random_on_device_kernel<<<1,1>>>(M);

    return M;
}

/* Count the number of rows and columns in a csv files (without headers) */
void count_elements_in_csv(const char* filename, int* rows, int* cols) {
  // Note: for the sake of convenience, we use some C++ functions here
  using namespace std;

  *rows = 0;
  *cols = 0;
  string row_as_string;
  string value;
  ifstream infile;
  infile.open(filename, ifstream::in);
	if (infile.is_open())
  {
    while (getline(infile, row_as_string, '\n')) {
				istringstream line_stream(row_as_string);
        int tempcols = 0;
        while (getline(line_stream, value, ',')) {
          ++tempcols;
        }
        if (tempcols > *cols) {
           *cols = tempcols;
        }
        ++(*rows);
			}
		infile.close();
	}
	else cout << "Cannot open file." << endl;
}

/** Read the data from a csv file into an fmatrix on the host.
 *  Careful: We assume that the matrix has the right dimensions!
 *  Use count_elements_in_csv(...) to get the dimensions if
 *  unknown.
 */
void fmatrix_fill_from_csv(fmatrix h_M,const char* filename) {
  // Note: for the sake of convenience, we use some C++ functions here
  using namespace std;
  string row_as_string;
  string value;
  ifstream infile;
  infile.open(filename, ifstream::in);
  int row = 0;
	if (infile.is_open())
  {
    while (getline(infile, row_as_string, '\n')) {
				istringstream line_stream(row_as_string);
        int col = 0;
        while (getline(line_stream, value, ',')) {
					getfm(h_M,row,col) = strtod(value.c_str(), NULL); 
          ++col;
				}
        ++row;
			}
		infile.close();
	}
	else cout << "Cannot open file." << endl;
}

fmatrix fmatrix_device_from_csv(const char* filename) {
  // first read the file to count the number of elements
  int rows = 0;
  int cols = 0;
  count_elements_in_csv(filename,&rows,&cols);

  // allocate the matrix on the host
  fmatrix h_M = fmatrix_create_on_host(rows,cols);

  // read the data into the host matrix
  fmatrix_fill_from_csv(h_M,filename);

  // copy the matrix to the device
  fmatrix M = fmatrix_copy_to_device(h_M);
  
  // destroy the host matrix
  fmatrix_free_on_host(&h_M);

  return M;
}




Overwriting fmatrix.cu


# Writing Matrix Functions with fmatrix
We will look at two examples that illustrate how to create, manipulate and compute using the fmatrix structure.
The first example is the matrix transpose, which generates a new matrix.
The second example is the matrix multiplication. It overwrites the data of an existing matrix with the result of the multiplication.

We start by looking at the code. Testing the code will be discussed in the section that comes afterwards.

## Matrix Addition


In [107]:
%%writefile fmatrix_add.cu
#include "fmatrix.cuh"
#include <assert.h>

#define THREADS_PER_BLOCK 1024
__global__ 
void fmatrix_add_kernel(fmatrix P,float a,fmatrix Y) {
    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / P.rows;
    int i = idx % P.rows;
    if (i < P.rows && j < P.cols ){
        getfm(P,i,j) += a*getfm(Y,i,j);
    }
}

/** Compute P = P + a*Y */
void fmatrix_add(fmatrix P,float a,fmatrix Y) {
    fmatrix_assert(P);
    fmatrix_assert(Y);
    assert(P.rows == Y.rows);
    assert(P.cols == Y.cols);
    int threadsPerBlock = fmatrix_elements(P);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_add_kernel<<< blocksPerGrid, threadsPerBlock >>>(P,a,Y);
    gpuErrchk( cudaPeekAtLastError() );
}

Overwriting fmatrix_add.cu


Note that the implementation for add doesn't include any synchronization. This is so that you can launch adds in parallel, which is perfectly fine if the date for both tasks is independent. Here's a small example:
```
// Without waiting:
fmatrix_add(P1,a,Y1);
fmatrix_add(P2,a,Y2);
deviceSynchronize();

// Waiting between each
fmatrix_add(P1,a,Y1);
deviceSynchronize();
fmatrix_add(P2,a,Y2);
deviceSynchronize();
```

## Matrix Multiplication

Here, we will create a new matrix A that contains the product of two matrices B and C.


In [108]:
%%writefile fmatrix_mult.cu
#include "fmatrix.cuh"
#include <assert.h>

#define THREADS_PER_BLOCK 1024

__global__
void fmatrix_multiplication_kernel(fmatrix A, float f, fmatrix B, fmatrix C) {
    // Each thread multiplies one row of B with one column of C
    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / A.rows;
    int i = idx % A.rows;
    if (i < A.rows && j < A.cols ){
        getfm(A,i,j) = 0.0;
        for (int k = 0; k < B.cols; ++k) {
          getfm(A,i,j) += f*getfm(B,i,k)*getfm(C,k,j);
        }
    }
}

/* Compute A = f*B*C */
void fmatrix_mult(fmatrix A, float f, fmatrix B, fmatrix C) {
    // First let's check for errors in the argument M.
    // This can help a LOT when debugging.
    // A,B,C need to have nonzero pointers etc.
    // fmatrix_assert(A);
    // fmatrix_assert(B);
    // fmatrix_assert(C);
    assert(A.rows == B.rows);
    assert(A.cols == C.cols);
    assert(B.cols == C.rows);
    
    // take one thread per element, and distribute
    // over as many blocks as necessary given
    // the hardware limit on the number of threads per block
    int threadsPerBlock = fmatrix_elements(A);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_multiplication_kernel<<< blocksPerGrid, threadsPerBlock >>>(A,f,B,C);
    // check for errors
    gpuErrchk( cudaPeekAtLastError() );
    // wait for the kernel to finish
    device_synchronize();
}



Overwriting fmatrix_mult.cu


## Simple Softmax

We use __shared__ float sum[P.cols] to store the sum of each column, each sum is only calculated once and then stored.

In [109]:
%%writefile fmatrix_simple_softmax.cu
#include "fmatrix.cuh"
#include <assert.h>

#define THREADS_PER_BLOCK 1024
__global__ 
void fmatrix_simple_softmax_kernel(fmatrix P, fmatrix Z) {
    
    __shared__ float sum[32];   // set max (P.cols) to 32

    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / P.rows;
    int i = idx % P.rows;
    if (i < P.rows && j < P.cols ){
        if (i == 0){
            sum[j] = 0;
            for (int k = 0; k < P.rows; ++k) {
              sum[j] += exp(getfm(P,k,j));
            }
        }
    }
    __syncthreads(); 
    if (i < P.rows && j < P.cols ){
        getfm(Z,i,j) = exp(getfm(P,i,j)) / sum[j];
    }
}

void fmatrix_simple_softmax(fmatrix P, fmatrix Z) {
    fmatrix_assert(P);
    fmatrix_assert(Z);
    assert(P.rows == Z.rows);
    assert(P.cols == Z.cols);
    int threadsPerBlock = fmatrix_elements(Z);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_simple_softmax_kernel<<< blocksPerGrid, threadsPerBlock >>>(P, Z);
    gpuErrchk( cudaPeekAtLastError() );
}

Overwriting fmatrix_simple_softmax.cu


##Stable Softmax

We use __shared__ float sum[P.cols] to store the sum of each column, each sum is only calculated once and then stored. Same idea for __shared__ float max[].

In [110]:
%%writefile fmatrix_stable_softmax.cu
#include "fmatrix.cuh"
#include <assert.h>

#define THREADS_PER_BLOCK 1024
__global__ 
void fmatrix_stable_softmax_kernel(fmatrix P, fmatrix Z) {
    
    __shared__ float sum[32];
    __shared__ float max[32];

    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / P.rows;
    int i = idx % P.rows;
    if (i < P.rows && j < P.cols ){
        if (i == 0){
            max[j] = getfm(P,0,j);
            for (int k = 0; k < P.rows; ++k) {
                if (getfm(P,k,j) > max[j]) max[j] = getfm(P,k,j);
            }
            sum[j] = 0;
            for (int k = 0; k < P.rows; ++k) {
                sum[j] += exp(getfm(P,k,j) - max[j]);
            }
        }
    }
    __syncthreads();


    if (i < P.rows && j < P.cols ){
        getfm(Z,i,j) = exp(getfm(P,i,j) - max[j]) / sum[j];
    }
}

void fmatrix_stable_softmax(fmatrix P, fmatrix Z) {
    fmatrix_assert(P);
    fmatrix_assert(Z);
    assert(P.rows == Z.rows);
    assert(P.cols == Z.cols);
    int threadsPerBlock = fmatrix_elements(Z);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_stable_softmax_kernel<<< blocksPerGrid, threadsPerBlock >>>(P, Z);
    gpuErrchk( cudaPeekAtLastError() );
}

Overwriting fmatrix_stable_softmax.cu


## Compute mu and sigma

We use shared float sum[P.rows] to store the sum of each row, each sum is only calculated once and then stored. Same idea for shared float sigma.

 compute the vectors 𝜇 and 𝜎 for a matrix X, the output is 2 matrix of size (X.rows,1), representing 𝜇 and 𝜎

In [111]:
%%writefile fmatrix_compute.cu
#include "fmatrix.cuh"
#include <assert.h>
#include <math.h>

#define THREADS_PER_BLOCK 1024
__global__ 
void fmatrix_compute_kernel(fmatrix P, fmatrix M_mu, fmatrix M_sigma) {
    __shared__ float sum[32];
    __shared__ float sigma[32];

    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / P.rows;
    int i = idx % P.rows;
    if (i < P.rows && j < P.cols ){
        if (j == 0){
            for (int k = 0; k < P.cols; ++k) {
              sum[i] += getfm(P,i,k);
            }   
        }
        __syncthreads(); 

        if (j == 0){
            float mu = sum[i] / P.cols;
            float sum_square = 0;     
            for (int k = 0; k < P.cols; ++k) {
              sum_square += pow(getfm(P,i,k) - mu, 2);
            }     
            sigma[i] = sqrt(sum_square / P.cols);
        }
        __syncthreads(); 
        getfm(M_mu,i,j) = sum[i] / P.cols;
        getfm(M_sigma,i,j) = sigma[i];

    }
}

void fmatrix_compute(fmatrix P, fmatrix mu, fmatrix sigma) {
    int threadsPerBlock = fmatrix_elements(P);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_compute_kernel<<< blocksPerGrid, threadsPerBlock >>>(P, mu, sigma);
    gpuErrchk( cudaPeekAtLastError() );
}

Overwriting fmatrix_compute.cu


## Normalize another matrix with given mu and sigma

In [112]:
%%writefile fmatrix_normalize.cu
#include "fmatrix.cuh"
#include <assert.h>
#include <math.h>

#define THREADS_PER_BLOCK 1024
__global__ 
void fmatrix_normalize_kernel(fmatrix P, fmatrix M_mu, fmatrix M_sigma) {
    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / P.rows;
    int i = idx % P.rows;
    if (i < P.rows && j < P.cols ){
        getfm(P, i, j) = (getfm(P, i, j) - getfm(M_mu, i, 1)) / getfm(M_sigma, i, 1);
    }
}

void fmatrix_normalize(fmatrix P, fmatrix mu, fmatrix sigma) {
    int threadsPerBlock = fmatrix_elements(P);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_normalize_kernel<<< blocksPerGrid, threadsPerBlock >>>(P, mu, sigma);
    gpuErrchk( cudaPeekAtLastError() );
}

Overwriting fmatrix_normalize.cu


# Testing the Matrix Functions

## Test my simple softmax

The range of possible values for the resulting elements of Z, is (0, 1) if they are computed perfectly. When the absolute value of elements of Z is not too big, our algorithm works well. However, if they are too large, then the results of exp operation will be out of bounds in programming, making it impossible to calculate, which is why we need to create a stable softmax later.

In [113]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_simple_softmax.cu"
#include "fmatrix_add.cu"

int main() {
    
    fmatrix M = fmatrix_create_random_on_device(3,5);
    printf("original matrix:\n");
    fmatrix_device_print(M);
    fmatrix_simple_softmax(M, M);
    printf("simple softmax:\n");
    fmatrix_device_print(M);
    
    

    
    fmatrix M1 = fmatrix_create_random_on_device(3,5);
    printf("large matrix:\n");
    fmatrix_add(M1,10000.0,M1);
    fmatrix_device_print(M1);
    fmatrix_simple_softmax(M1, M1);
    printf("simple softmax large :\n");
    fmatrix_device_print(M1);


    fmatrix_free_on_device(&M);
    fmatrix_free_on_device(&M1);
}

Overwriting test.cu


In [114]:
!nvcc -arch=sm_35 -Wno-deprecated-gpu-targets -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [115]:
!./a.out

original matrix:
[
0.480439,	-0.686023,	-0.544713,	-0.624402,	0.949322;
-0.123098,	-0.857265,	-0.341283,	0.830767,	0.094939;
0.034025,	-0.074983,	-0.711869,	0.080250,	0.306321
]
simple softmax:
[
0.457291,	0.271373,	0.325556,	0.136830,	0.512490;
0.250080,	0.228664,	0.399001,	0.586344,	0.218088;
0.292629,	0.499963,	0.275442,	0.276826,	0.269422
]
large matrix:
[
4804.867676,	-6860.913574,	-5447.679688,	-6244.642578,	9494.166992;
-1231.100220,	-8573.507812,	-3413.171631,	8308.500000,	949.480103;
340.287140,	-749.907410,	-7119.405273,	802.576904,	3063.511963
]
simple softmax large :
[
nan,	nan,	nan,	0.000000,	nan;
0.000000,	nan,	nan,	nan,	nan;
nan,	nan,	nan,	nan,	nan
]


When the absolute value of elements of Z is not too big, our algorithm works well. However, if they are too large, then the results of exp operation will be out of bounds, making it impossible to calculate, which is why we need to create a stable softmax next.

## Test my stable softmax

The stable softmax is identical to softmax over the real numbers, but is more stable in floating point since it avoids overflow and improves the precision. Now, when the elements of Z are too large, the results of exp operation will not be out of bounds, and it's possible to calculate.

In [116]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_stable_softmax.cu"
#include "fmatrix_add.cu"

int main() {
    
    fmatrix M = fmatrix_create_random_on_device(3,5);
    fmatrix_device_to_csv("matrix_stable_softmax_test.csv",M);
    printf("original matrix:\n");
    fmatrix_device_print(M);
    fmatrix_stable_softmax(M, M);
    printf("stable softmax:\n");
    fmatrix_device_print(M);
    fmatrix_device_to_csv("result_matrix_stable_softmax_test.csv",M);


    
    fmatrix M1 = fmatrix_create_random_on_device(3,5);
    fmatrix_device_to_csv("large_matrix_stable_softmax_test.csv",M1);
    printf("large matrix:\n");
    fmatrix_add(M1,50.0,M1);
    fmatrix_device_print(M1);
    fmatrix_stable_softmax(M1, M1);
    printf("stable softmax large:\n");
    fmatrix_device_print(M1);
    fmatrix_device_to_csv("result_large_matrix_stable_softmax_test.csv",M1);


    fmatrix_free_on_device(&M);
    fmatrix_free_on_device(&M1);

}

Overwriting test.cu


In [117]:
!nvcc -arch=sm_35 -Wno-deprecated-gpu-targets -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [118]:
!./a.out

original matrix:
[
0.480439,	-0.686023,	-0.544713,	-0.624402,	0.949322;
-0.123098,	-0.857265,	-0.341283,	0.830767,	0.094939;
0.034025,	-0.074983,	-0.711869,	0.080250,	0.306321
]
stable softmax:
[
0.457291,	0.271373,	0.325556,	0.136830,	0.512490;
0.250080,	0.228664,	0.399001,	0.586344,	0.218088;
0.292629,	0.499963,	0.275442,	0.276826,	0.269422
]
large matrix:
[
24.502373,	-34.987164,	-27.780390,	-31.844492,	48.415409;
-6.277984,	-43.720516,	-17.405436,	42.369114,	4.841865;
1.735291,	-3.824145,	-36.305336,	4.092733,	15.622348
]
stable softmax large:
[
1.000000,	0.000000,	0.000031,	0.000000,	1.000000;
0.000000,	0.000000,	0.999969,	1.000000,	0.000000;
0.000000,	1.000000,	0.000000,	0.000000,	0.000000
]


## Test my compute mu and sigma

In [119]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_compute.cu"
#include "fmatrix_normalize.cu"
#include "fmatrix_add.cu"

int main() {
    
    fmatrix P = fmatrix_create_random_on_device(3,5);
    fmatrix Q = fmatrix_create_random_on_device(3,5);
    fmatrix_add(Q,0.2,Q);

    fmatrix mu = fmatrix_create_random_on_device(3,1);
    fmatrix sigma = fmatrix_create_random_on_device(3,1);

    printf("original matrix:\n");
    fmatrix_device_print(P);
    fmatrix_device_to_csv("matrix_normalize.csv",P);
    
    fmatrix_compute(P, mu, sigma);
    printf("mu:\n");
    fmatrix_device_to_csv("matrix_mu.csv",mu);
    fmatrix_device_print(mu);
    printf("sigma:\n");
    fmatrix_device_to_csv("matrix_sigma.csv",sigma);
    fmatrix_device_print(sigma);
    

    printf("second matrix which is to be normalized:\n");
    fmatrix_device_print(Q);
    fmatrix_device_to_csv("matrix_to_be_normalized.csv",Q);
    fmatrix_normalize(Q, mu, sigma);
    printf("after normalization:\n");
    fmatrix_device_to_csv("matrix_after_normalize.csv",Q);
    fmatrix_device_print(Q);

    fmatrix_free_on_device(&P);
    fmatrix_free_on_device(&mu);
    fmatrix_free_on_device(&sigma);
    fmatrix_free_on_device(&Q);


}

Overwriting test.cu


In [120]:
!nvcc -arch=sm_35 -Wno-deprecated-gpu-targets -g -G -I /usr/local/cuda/samples/common/inc/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [121]:
!./a.out

original matrix:
[
0.480439,	-0.686023,	-0.544713,	-0.624402,	0.949322;
-0.123098,	-0.857265,	-0.341283,	0.830767,	0.094939;
0.034025,	-0.074983,	-0.711869,	0.080250,	0.306321
]
mu:
[
-0.085076;
-0.079188;
-0.073251
]
sigma:
[
0.671277;
0.553961;
0.342631
]
second matrix which is to be normalized:
[
0.576526,	-0.823227,	-0.653656,	-0.749282,	1.139186;
-0.147717,	-1.028718,	-0.409540,	0.996920,	0.113926;
0.040830,	-0.089980,	-0.854243,	0.096300,	0.367585
]
after normalization:
[
0.985588,	-1.099624,	-0.847014,	-0.989468,	1.823781;
-0.123708,	-1.714074,	-0.596345,	1.942571,	0.348606;
0.332958,	-0.048824,	-2.279393,	0.494850,	1.286619
]


## My own comparison between my stable softmax and Numpy

Exactly the same results with numpy, created functions for softmax are correct.

In [131]:
import numpy as np
# here the softmax is row-major, so we should (1)m = m.T, (2)m = softmax(m), (3) m = m.T

def softmax(x):
    x_row_max = x.max(axis=-1)
    x_row_max = x_row_max.reshape(list(x.shape)[:-1]+[1])
    x = x - x_row_max
    x_exp = np.exp(x)
    x_exp_row_sum = x_exp.sum(axis=-1).reshape(list(x.shape)[:-1]+[1])
    softmax = x_exp / x_exp_row_sum
    return softmax

m = np.loadtxt("matrix_stable_softmax_test.csv",delimiter=',',ndmin=2)
print("Test our stable softmax:")
print("\ninitial matrix:")
print(m)

m = np.transpose(m)
m = softmax(m)
m = np.transpose(m)
print("\nnumpy softmax result:")
print(m)

my_m = np.loadtxt("result_matrix_stable_softmax_test.csv",delimiter=',',ndmin=2)
print("\nmy stable softmax matrix:")
print(my_m)

result = np.allclose(m, my_m)
print("\nTest passed?", result)



Test our stable softmax:

initial matrix:
[[ 0.48043871 -0.68602276 -0.5447135  -0.62440181  0.94932175]
 [-0.12309772 -0.857265   -0.34128302  0.83076692  0.09493852]
 [ 0.03402531 -0.07498324 -0.71186936  0.08024967  0.30632055]]

numpy softmax result:
[[0.45729057 0.27137315 0.32555605 0.13682983 0.51248993]
 [0.25008043 0.22866374 0.39900148 0.58634407 0.21808782]
 [0.292629   0.49996311 0.27544247 0.2768261  0.26942225]]

my stable softmax matrix:
[[0.45729053 0.27137315 0.32555604 0.13682982 0.51248991]
 [0.25008041 0.22866376 0.39900148 0.58634406 0.21808782]
 [0.292629   0.4999631  0.27544248 0.27682611 0.26942223]]

Test passed? True


## My own comparison between my normalization and Numpy

Exactly the same results with numpy, created functions for normalization are correct.

In [136]:
import numpy as np
# here the softmax is row-major, so we should (1)m = m.T, (2)m = softmax(m), (3) m = m.T

def compute(x):
    x_row_sum = x.sum(axis=-1).reshape(list(x.shape)[:-1]+[1])
    x_row_mean = x_row_sum / x.shape[1]
    mu = x_row_mean
    x_row_square = np.square(x - x_row_mean)
    x_row_square_sum = x_row_square.sum(axis=-1).reshape(list(x.shape)[:-1]+[1])
    sigma = np.sqrt( x_row_square_sum / x.shape[1])
    return mu, sigma

def normalize(x, mu, sigma):
    x = (x - mu) / sigma
    return x

print("Test our normalization:\n")

m = np.loadtxt("matrix_normalize.csv",delimiter=',',ndmin=2)
print("fist matrix(used to calculate mu and sigma):")
print(m)

q = np.loadtxt("matrix_to_be_normalized.csv",delimiter=',',ndmin=2)
print("second matrix(to be normalized):")
print(q)

mu, sigma = compute(m)
print("\nnumpy compute result:")
print("mu of first matrix:")
print(mu)
print("sigma of first matrix:")
print(sigma)
print("applying normalization on the secon matrix using mu and sigma of the first matrix:")
q_n = normalize(q, mu, sigma)
print(q_n)


print("\nmy normalization function result:")
mu_1 = np.loadtxt("matrix_mu.csv",delimiter=',',ndmin=2)
print("mu of first matrix:")
print(mu_1)
sigma_1 = np.loadtxt("matrix_sigma.csv",delimiter=',',ndmin=2)
print("sigma of first matrix:")
print(sigma_1)
q_n_1 = np.loadtxt("matrix_after_normalize.csv",delimiter=',',ndmin=2)
print("applying normalization on the secon matrix using mu and sigma of the first matrix:")
print(q_n_1)



result1 = np.allclose(mu, mu_1)
print("\nmu test passed?", result1)
result2 = np.allclose(sigma, sigma_1)
print("sigma test passed?", result2)
result3 = np.allclose(q_n, q_n_1)
print("second matrix after normalize test passed?", result3)



Test our normalization:

fist matrix(used to calculate mu and sigma):
[[ 0.48043871 -0.68602276 -0.5447135  -0.62440181  0.94932175]
 [-0.12309772 -0.857265   -0.34128302  0.83076692  0.09493852]
 [ 0.03402531 -0.07498324 -0.71186936  0.08024967  0.30632055]]
second matrix(to be normalized):
[[ 0.57652646 -0.82322729 -0.65365618 -0.74928218  1.13918614]
 [-0.14771727 -1.02871799 -0.40953964  0.99692029  0.11392622]
 [ 0.04083037 -0.08997989 -0.85424322  0.0962996   0.36758465]]

numpy compute result:
mu of first matrix:
[[-0.08507552]
 [-0.07918806]
 [-0.07325141]]
sigma of first matrix:
[[0.67127663]
 [0.55396095]
 [0.34263147]]
applying normalization on the secon matrix using mu and sigma of the first matrix:
[[ 0.9855877  -1.09962381 -0.84701394 -0.98946788  1.82378116]
 [-0.12370765 -1.71407377 -0.59634452  1.94257077  0.3486063 ]
 [ 0.3329577  -0.04882351 -2.27939307  0.49484951  1.2866187 ]]

my normalization function result:
mu of first matrix:
[[-0.08507552]
 [-0.07918806]
 [-0